In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [4]:
data = pd.read_csv('mini.csv')
data.head()

,customer_id,occurence,cost,item_id
0,416705,2017-05-07 21:58:10.000000,299.0,515274
1,13891,2018-02-10 17:35:11.000000,1090.0,828115
2,9081,2017-12-21 17:13:44.000000,499.0,695501
3,470904,2017-10-31 10:39:49.000000,290.0,899821
4,58500,2018-03-09 20:57:29.000000,150.0,518554


In [9]:
data['occurence'] = data['occurence'].apply(pd.to_datetime)

In [10]:
data.head()

,customer_id,occurence,cost,item_id
0,416705,2017-05-07 21:58:10,299.0,515274
1,13891,2018-02-10 17:35:11,1090.0,828115
2,9081,2017-12-21 17:13:44,499.0,695501
3,470904,2017-10-31 10:39:49,290.0,899821
4,58500,2018-03-09 20:57:29,150.0,518554


In [34]:
hz = pd.DataFrame(data.groupby('customer_id')['cost'].mean())
hz['number of transactions'] = data.groupby('customer_id')['cost'].count()
hz['total'] = data.groupby('customer_id') ['cost'].sum()

In [35]:
data['year'] = pd.to_datetime(data.occurence).dt.year
data['month'] = pd.to_datetime(data.occurence).dt.month
data = data[data['year'] == 2018]
five_month = data[data['month'] <= 5]
target_month = data[data['month'] == 6]

In [48]:
target_month = target_month[['customer_id']].drop_duplicates(subset = ['customer_id'], keep = False)
five_month = five_month[['customer_id']].drop_duplicates(subset = ['customer_id'],keep = False)
target_month['target'] = 'yes'
result = pd.merge(five_month, target_month, how = 'left', on = ['customer_id'])
result['target'] = (result.target.isnull()).astype('int')

In [86]:
df_total = pd.merge(result,hz, how = 'inner', on = ['customer_id'])
df_total.head()

,customer_id,target,cost,number of transactions,total
0,568402,1,590.000000,1,590.0
1,530299,1,500.000000,1,500.0
2,112048,1,1390.000000,1,1390.0
3,561707,1,1346.666667,3,4040.0
4,560111,1,3890.000000,1,3890.0


In [109]:
df_total.columns

Index(['target', 'cost', 'number of transactions', 'total'], dtype='object')

In [110]:
df_total.head()

,target,cost,number of transactions,total
0,1,590.000000,1,590.0
1,1,500.000000,1,500.0
2,1,1390.000000,1,1390.0
3,1,1346.666667,3,4040.0
4,1,3890.000000,1,3890.0


In [112]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [119]:
X = df_total.drop('target', axis = 1)
y = df_total['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [123]:
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 17, max_depth=3, min_samples_leaf=5) 
clf_entropy.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [130]:
prediction = clf_entropy.predict(X_test)

In [134]:
prediction

array([1, 1, 1, ..., 0, 1, 1])

In [135]:
accuracy_score(y_test, prediction)

0.9452163844261109

In [136]:
tree_params = {'max_depth': np.arange(1, 11), 'max_features':[.5, .7, 1]}

In [137]:
tree_grid = GridSearchCV(clf_entropy, tree_params, cv=5)

In [139]:
tree_grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), 'max_features': [0.5, 0.7, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [140]:
tree_grid.best_score_, tree_grid.best_params_ 

(0.9490090257746348, {'max_depth': 3, 'max_features': 0.7})

# THE END